# **Nhiệm vụ 4 – Merge dữ liệu**

*Mục tiêu : tạo một file dữ liệu hoàn chỉnh phục vụ phân tích sâu hơn*

In [2]:
# import pandas
import pandas as pd

# Read csv
df_teams = pd.read_csv(f'./data_clean/team_info_clean.csv')
df_players = pd.read_csv(f'./data_clean/player_stats_clean.csv')
df_matches = pd.read_csv(f'./data_clean/match_results_clean.csv', parse_dates=['match_date'])

**1. Merge player_stats với team_info theo team_id để lấy thêm thông tin đội và thành phố**

In [8]:


print("1. Merge Player Stats với Team Info")
players_merge = pd.merge(
    df_players,
    df_teams[['team_id', 'team_name', 'home_city']],
    on='team_id',
    how='left'
)
players_merge.head()

1. Merge Player Stats với Team Info


,player_id,player_name,team_id,position,goals,team_name,home_city
0,P001,Nguyen Van A,T01,MF,1,Clb Thanh Pho Hcm,Ha Noi
1,P002,Le Cong Vinh,T01,FW,2,Clb Thanh Pho Hcm,Ha Noi
2,P003,Pham Thanh Luan,T01,MF,0,Clb Thanh Pho Hcm,Ha Noi
3,P004,Le Cong Vinh,T01,DF,0,Clb Thanh Pho Hcm,Ha Noi
4,P005,Le Cong Vinh,T01,DF,2,Clb Thanh Pho Hcm,Ha Noi


**2. Merge match_results với team_info để gắn tên đội cho home_team_id và away_team_id**

In [15]:
match_home = df_matches.merge(
    df_teams[['team_id', 'team_name', 'home_city']].add_prefix("home_"),
    left_on="home_team_id",
    right_on="home_team_id",
    how="left"
)
match_home.head()

,match_id,home_team_id,away_team_id,home_goals,away_goals,match_date,home_team_name,home_home_city
0,M001,T07,T07,3,3,2024-04-24,Clb Binh Duong,Ha Noi
1,M002,T03,T07,2,3,2024-01-08,Clb Ha Noi,Ha Noi
2,M003,T07,T05,4,1,2024-04-13,Clb Binh Duong,Ha Noi
3,M004,T02,T01,2,0,2024-05-08,Clb Hai Phong,Thanh pho Hcm
4,M005,T05,T06,3,2,2024-06-19,Clb Thanh Pho Hcm,Ha Noi


In [19]:
match_full = match_home.merge(
    df_teams[['team_id', 'team_name', 'home_city']].add_prefix("away_"),
    left_on="away_team_id",
    right_on="away_team_id",
    how="left"
)
match_full.head()

,match_id,home_team_id,away_team_id,home_goals,away_goals,match_date,home_team_name,home_home_city,away_team_name,away_home_city
0,M001,T07,T07,3,3,2024-04-24,Clb Binh Duong,Ha Noi,Clb Binh Duong,Ha Noi
1,M002,T03,T07,2,3,2024-01-08,Clb Ha Noi,Ha Noi,Clb Binh Duong,Ha Noi
2,M003,T07,T05,4,1,2024-04-13,Clb Binh Duong,Ha Noi,Clb Thanh Pho Hcm,Ha Noi
3,M004,T02,T01,2,0,2024-05-08,Clb Hai Phong,Thanh pho Hcm,Clb Thanh Pho Hcm,Ha Noi
4,M005,T05,T06,3,2,2024-06-19,Clb Thanh Pho Hcm,Ha Noi,Clb Ha Noi,Da Nang


**3. Phát hiện các player_stats không khớp với team_info (team_id không tồn tại)**

In [20]:
mismatched_players = players_merge[players_merge['team_name'].isnull()]

if mismatched_players.empty:
    print("-> Không phát hiện cầu thủ nào có team_id không khớp với team_info.")
else:
    print(f"-> Phát hiện {len(mismatched_players)} bản ghi cầu thủ không khớp:")
    print(mismatched_players)

-> Không phát hiện cầu thủ nào có team_id không khớp với team_info.


**4. Phát hiện các match_results có team_id không tìm thấy trong team_info**

In [24]:
# Các bản ghi không khớp sẽ có 'home_team_name' hoặc 'away_team_name' là NaN
# Kiểm tra xem cột tên đội (đã merge) có bị thiếu (NaN) hay không

mismatched_matches = match_full[
    match_full['home_team_name'].isnull() | match_full['away_team_name'].isnull()
]

if mismatched_matches.empty:
    print("-> Không phát hiện trận đấu nào có home_team_id hoặc away_team_id không khớp.")
else:
    print(f"-> Phát hiện {len(mismatched_matches)} bản ghi trận đấu không khớp:")
    print(mismatched_matches[['match_id', 'home_team_id', 'away_team_id', 'home_team_name', 'away_team_name']])

-> Không phát hiện trận đấu nào có home_team_id hoặc away_team_id không khớp.
